# Approach

In [16]:
from sklearn import tree, preprocessing, ensemble, metrics, svm
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC

from sklearn.datasets import make_classification
from sklearn.utils import resample
from sklearn.model_selection import cross_validate, cross_val_predict, KFold, cross_val_score, train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import importlib

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_tree
from graphviz import Digraph

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

In [17]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMinimize, LpInteger
from ortools.linear_solver import pywraplp
from cvxopt import glpk
import math

# Load data
demand_history = pd.read_csv("Demand_History.csv")
existing_ev_infrastructure = pd.read_csv("exisiting_EV_infrastructure_2018.csv")

In [18]:
existing_ev_infrastructure

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS
0,0,50.163110,19.412014,23,5,3
1,1,37.336451,58.119225,27,4,7
2,2,46.709232,57.525650,31,6,14
3,3,30.528626,55.379835,26,5,5
4,4,51.521781,35.116755,32,11,6
...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4
96,96,30.318396,33.388335,32,5,10
97,97,36.218839,22.235766,32,4,14
98,98,42.936915,38.122442,28,7,5


In [19]:
demand_points = demand_history.iloc[:, 3:]

# Transform the data for time series prediction
years = list(demand_points.columns.astype(int))
X, y = [], []

for i, row in demand_points.iterrows():
    for j in range(len(years) - 1):
        X.append([demand_history.loc[i, 'x_coordinate'], demand_history.loc[i, 'y_coordinate']] + [years[j]] + row[j:j + 1].tolist())  # x_coordinate, y_coordinate, year, demand
        y.append(row[j + 1])  # demand of the next year

In [20]:
X = np.array(X)
y = np.array(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=3, learning_rate=0.1)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [21]:
# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Predict demands for 2019 and 2020
years_to_predict = [2019, 2020]
predictions = []

for i, row in demand_points.iterrows():
    demands = []
    for year in years_to_predict:
        pred_demand = model.predict(np.array([[demand_history.loc[i, 'x_coordinate'], demand_history.loc[i, 'y_coordinate'], year, row.iloc[-1]]]))[0]
        demands.append(pred_demand)
        row = row.shift(-1)
        row.iloc[-1] = pred_demand
    predictions.append(demands)

predictions_df = pd.DataFrame(predictions, columns=["2019", "2020"])
final_df = pd.concat([demand_history.iloc[:, :3], demand_points, predictions_df], axis=1)

Mean Squared Error: 58.43348617106841


In [22]:
demand_history = final_df[:]
demand_history

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,18.349737,25.217339
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,16.824572,23.453083
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,20.044586,27.821360
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,22.027550,29.114857
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,22.601883,30.470375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4091,59.5,63.5,0.171015,0.334565,0.556055,1.373291,1.837586,2.517146,3.352280,4.149888,5.426193,8.220200,13.009814
4092,4092,60.5,63.5,0.041716,0.061741,0.131291,0.386540,0.755846,0.941116,1.107797,1.309479,2.057450,2.959660,4.269553
4093,4093,61.5,63.5,0.100895,0.180352,0.296299,0.705373,1.300220,1.608609,1.822806,2.333681,3.218519,4.269553,6.631501
4094,4094,62.5,63.5,0.155353,0.290825,0.557803,1.516066,2.399426,2.719197,4.494515,6.096858,6.262574,9.414526,13.855034


In [23]:
import pulp

# Constants
a, b, c = 1, 25, 600
r = 1.5
cap_scs, cap_fcs = 200, 400
n_demand_points, n_supply_points = 4096, 100
years = [2019, 2020]

# Distance function (assuming Euclidean distance)
def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

# Calculate the distance matrix
dist_matrix = np.zeros((n_demand_points, n_supply_points))
for i, demand_point in demand_history.iterrows():
    for j, supply_point in existing_ev_infrastructure.iterrows():
        dist_matrix[i, j] = distance(demand_point.x_coordinate, demand_point.y_coordinate,
                                     supply_point.x_coordinate, supply_point.y_coordinate)

# Optimization
results = []
for year in years:
    # Create a problem instance
    prob = pulp.LpProblem(f"EV_Charging_Infrastructure_Optimization_{year}", pulp.LpMinimize)

    # Define variables
    DS = pulp.LpVariable.dicts("DS", ((i, j) for i in range(n_demand_points) for j in range(n_supply_points)), lowBound=0)
    SCS = pulp.LpVariable.dicts("SCS", range(n_supply_points), lowBound=0, cat='Integer')
    FCS = pulp.LpVariable.dicts("FCS", range(n_supply_points), lowBound=0, cat='Integer')

    # Add constraints to the problem
    for i in range(n_demand_points):
        prob += pulp.lpSum(DS[i, j] for j in range(n_supply_points)) == demand_history.loc[i, str(year)]

    for j in range(n_supply_points):
        prob += pulp.lpSum(DS[i, j] for i in range(n_demand_points)) <= (cap_scs * SCS[j]) + (cap_fcs * FCS[j])
        prob += SCS[j] + FCS[j] <= existing_ev_infrastructure.loc[j, "total_parking_slots"]
        prob += SCS[j] >= existing_ev_infrastructure.loc[j, "existing_num_SCS"]
        prob += FCS[j] >= existing_ev_infrastructure.loc[j, "existing_num_FCS"]

    # Define the objective function
    cost_cd = pulp.lpSum(dist_matrix[i, j] * DS[i, j] for i in range(n_demand_points) for j in range(n_supply_points))
    cost_dm = pulp.lpSum(abs(demand_history.loc[i, str(year)] - demand_history.loc[i, str(year - 1)]) for i in range(n_demand_points))
    cost_if = pulp.lpSum(SCS[j] + r * FCS[j] for j in range(n_supply_points))

    prob += a * cost_cd + b * cost_dm + c * cost_if

    # Solve the optimization problem
    prob.solve()

    result = {
        "year": year,
        "DS": {(i, j): DS[i, j].varValue for i in range(n_demand_points) for j in range(n_supply_points)},
        "SCS": {j: int(math.ceil(SCS[j].varValue)) for j in range(n_supply_points)},
        "FCS": {j: int(math.ceil(FCS[j].varValue)) for j in range(n_supply_points)}
    }

    results.append(result)

In [24]:
result_rows = []

for result in results:
    year = result["year"]

    for supply_point_index in range(n_supply_points):
        result_rows.append({
            "year": year,
            "data_type": "SCS",
            "demand_point_index": np.nan,
            "supply_point_index": supply_point_index,
            "value": result["SCS"][supply_point_index]
        })

        result_rows.append({
            "year": year,
            "data_type": "FCS",
            "demand_point_index": np.nan,
            "supply_point_index": supply_point_index,
            "value": result["FCS"][supply_point_index]
        })

    for demand_point_index in range(n_demand_points):
        for supply_point_index in range(n_supply_points):
            result_rows.append({
                "year": year,
                "data_type": "DS",
                "demand_point_index": demand_point_index,
                "supply_point_index": supply_point_index,
                "value": result["DS"][(demand_point_index, supply_point_index)]
            })

results_df = pd.DataFrame(result_rows)
results_df

,year,data_type,demand_point_index,supply_point_index,value
0,2019,SCS,NaN,0,5.0
1,2019,FCS,NaN,0,18.0
2,2019,SCS,NaN,1,4.0
3,2019,FCS,NaN,1,7.0
4,2019,SCS,NaN,2,6.0
...,...,...,...,...,...
819595,2020,DS,4095.0,95,0.0
819596,2020,DS,4095.0,96,0.0
819597,2020,DS,4095.0,97,0.0
819598,2020,DS,4095.0,98,0.0


In [25]:
results_df['data_type'] = pd.Categorical(results_df['data_type'], categories=['SCS', 'FCS', 'DS'], ordered=True)
test = results_df.sort_values(['year', 'data_type', 'supply_point_index', 'demand_point_index']).reset_index(drop=True)
test

,year,data_type,demand_point_index,supply_point_index,value
0,2019,SCS,NaN,0,5.0
1,2019,SCS,NaN,1,4.0
2,2019,SCS,NaN,2,6.0
3,2019,SCS,NaN,3,5.0
4,2019,SCS,NaN,4,11.0
...,...,...,...,...,...
819595,2020,DS,4091.0,99,0.0
819596,2020,DS,4092.0,99,0.0
819597,2020,DS,4093.0,99,0.0
819598,2020,DS,4094.0,99,0.0


In [26]:
test.to_csv('SHELL_output.csv', index=False)

In [27]:
# Filter results_df for SCS and FCS values
scs_values = results_df.loc[results_df['data_type'] == 'SCS', ['year', 'supply_point_index', 'value']]
fcs_values = results_df.loc[results_df['data_type'] == 'FCS', ['year', 'supply_point_index', 'value']]

# Pivot the data to get SCS and FCS values for each year
scs_pivot = scs_values.pivot(index='supply_point_index', columns='year', values='value').reset_index()
fcs_pivot = fcs_values.pivot(index='supply_point_index', columns='year', values='value').reset_index()

# Merge the data with existing_ev_infrastructure
ev_infra_merged = existing_ev_infrastructure.merge(scs_pivot, on='supply_point_index').merge(fcs_pivot, on='supply_point_index', suffixes=('_SCS', '_FCS'))
ev_infra_merged

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,2019_SCS,2020_SCS,2019_FCS,2020_FCS
0,0,50.163110,19.412014,23,5,3,5.0,5.0,18.0,18.0
1,1,37.336451,58.119225,27,4,7,4.0,4.0,7.0,7.0
2,2,46.709232,57.525650,31,6,14,6.0,6.0,14.0,14.0
3,3,30.528626,55.379835,26,5,5,5.0,5.0,5.0,5.0
4,4,51.521781,35.116755,32,11,6,11.0,11.0,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,3.0,3.0,15.0,21.0
96,96,30.318396,33.388335,32,5,10,5.0,5.0,20.0,25.0
97,97,36.218839,22.235766,32,4,14,4.0,4.0,28.0,28.0
98,98,42.936915,38.122442,28,7,5,7.0,7.0,5.0,5.0


In [28]:
# Check if 2019_SCS + 2019_FCS > total_parking_slots
condition_2019 = ev_infra_merged['2019_SCS'] + ev_infra_merged['2019_FCS'] > ev_infra_merged['total_parking_slots']

# Check if 2020_SCS + 2020_FCS > total_parking_slots
condition_2020 = ev_infra_merged['2020_SCS'] + ev_infra_merged['2020_FCS'] > ev_infra_merged['total_parking_slots']

# Select rows that meet the conditions
result = ev_infra_merged[condition_2019 | condition_2020]

# Display the results
print(result)

Empty DataFrame
Columns: [supply_point_index, x_coordinate, y_coordinate, total_parking_slots, existing_num_SCS, existing_num_FCS, 2019_SCS, 2020_SCS, 2019_FCS, 2020_FCS]
Index: []


In [29]:
condition_2020.value_counts()

False    100
dtype: int64

In [30]:
condition_2019.value_counts()

False    100
dtype: int64